In [69]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import psycopg2 as ps

# get date for everyday
date = input(f"Enter date by follwing this format DD/MM/YYYY or MM/DD/YYYY or YYYY/MM/DD or YYYY/DD/MM: ")
# link for website to work
url = f"https://www.yallakora.com/match-center/?date={date}"
# load the website from the link
load_my_page = requests.get(url)


# the main function to do operations on loaded page
def main(load_my_page):
    # show content in my loaded page
    src = load_my_page.content
    # The code is formatted and displayed clearly
    soup = BeautifulSoup(src, "lxml")
    # list of ditc
    match_details = []
    # choose the div with the class matchCard to show all div has this name
    champion = soup.find_all("div", {"class": "matchCard"})

    # create a function to collect all information from the matchCard
    def get_match_info(champion):
        # get the h2 in matchCard
        # champion_title = champion.contents[0].isspace() #  line have a space
        # show h2
        champion_title = champion.contents[1].find("h2").text.strip()
        # get all match information from ul class
        all_matchs = champion.contents[3].find_all("div", {"class": "item"})
        # number of matches
        number_of_matchs = len(all_matchs)

        for i in range(number_of_matchs):
            # get teams name
            team_A = all_matchs[i].find("div", {"class": "teamA"}).text.strip()
            team_B = all_matchs[i].find("div", {"class": "teamB"}).text.strip()

            # get score
            score = (
                all_matchs[i]
                .find("div", {"class": "MResult"})
                .find_all("span", {"class": "score"})
            )
            score_text = f"{score[0].text.strip()} - {score[1].text.strip()}"
            # get match time
            match_time = (
                all_matchs[i]
                .find("div", {"class": "MResult"})
                .find("span", {"class": "time"})
                .text.strip()
            )

            # add match info to match details
            match_details.append(
                {
                    "matchTime": match_time,
                    "result": score_text,
                    "teamA": team_A,
                    "champion": champion_title,
                    "teamB": team_B,
                }
            )
    # loop in all champion
    for i in range(len(champion)):

        get_match_info(champion[i])
    # keys in match_details
    keys = match_details[0].keys()
    # use csv to create csv file
    with open("mtv1.csv", "w",encoding="utf-8") as file:
        dict_writer = csv.DictWriter(file, keys)
        # fill columns with values from match_details
        dict_writer.writeheader()
        # fill rows with values from match_details
        dict_writer.writerows(match_details)
        print("file created")
        # open file with pandas
    colums=['matchTime','result','teamB','teamA','champion']
    op_file = pd.DataFrame(match_details)
    # show head in my data
    match_data_list = op_file.astype('str').values.tolist()
    csv_for_file=pd.DataFrame(match_data_list,columns=colums)
    print(csv_for_file)
    
    # ETL
    conn = ps.connect(database="DB1", user="postgres", password="root", host="127.0.0.1", port="5432")
    print("data is connected")
    cur = conn.cursor()

    # create table
  
    cur.execute(
        """CREATE TABLE matchs
                    ( 
                    CHAMPION_TYPE VARCHAR(100),
                    TEAM_A VARCHAR(100),
                    TEAM_B VARCHAR(100),
                    TIME TEXT,
                    RESULT VARCHAR(100))""")
    print("matchs table has been created successfully")
   
    # will use ? sign to represent each column names inside VALUE().

    for index, row in csv_for_file.iterrows():
        insert_script = '''INSERT INTO matchs (CHAMPION_TYPE,TEAM_A,TEAM_B,TIME,RESULT) VALUES (%s, %s, %s, %s, %s)'''
        insert_value = (row['teamA'],row['champion'],row['teamB'],row['matchTime'],row['result'])
        cur.execute(insert_script,insert_value)
        
    print("inserted")
    conn.commit()
main(load_my_page)




file created
   matchTime result              teamB                teamA          champion
0      16:00  0 - 0        طلائع الجيش        الدوري المصري          الداخلية
1      19:00  2 - 0             المصري        الدوري المصري             سموحة
2      22:00  3 - 0       بايرن ميونيخ    دوري أبطال أوروبا            لاتسيو
3      22:00  1 - 2       ريال سوسيداد    دوري أبطال أوروبا  باريس سان جيرمان
4      12:00  1 - 1     تشونبك هيونداي      دوري أبطال آسيا    أولسان هيونداي
5      20:00  2 - 0             الهلال      دوري أبطال آسيا           الاتحاد
6      19:00  0 - 0      نهضة الزمامرة       الدوري المغربي   الوداد البيضاوي
7      15:30  1 - 2        مصر المقاصة  دوري القسم الثاني-أ       بترول أسيوط
8      15:30  2 - 1    أبو قير للاسمدة  دوري القسم الثاني-أ             مكادي
9      15:30  2 - 1       جمهورية شبين  دوري القسم الثاني-أ     النصر للتعدين
10     15:30  4 - 0              دكرنس  دوري القسم الثاني-أ            النجوم
11     15:30  1 - 0  المصرية للاتصالات  دوري القسم 